In [ ]:
pip install snscrape

In [ ]:
pip install --upgrade pip

In [ ]:
pip install pandas

In [ ]:
pip install textblob

In [ ]:
pip install wordcloud

In [ ]:
pip install seaborn

In [ ]:
pip install spacy

In [ ]:
pip install -U pip setuptools wheel

In [ ]:
pip install -U spacy


In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy download en_core_web_lg 

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
#Importing libraries 


from datetime import date
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from textblob import TextBlob  
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")

# 2. Scraping tweets 
<br>The code below scrapes 1000 tweets with the keyword ‘crypto’. The tweets list is created is converted into a data frame with the column name Tweets.<br>

In [ ]:
# Generating datetime objects
from datetime import datetime, timedelta
now = datetime.now()
now = now.strftime('%Y-%m-%d')
yesterday = datetime.now() - timedelta(days = 1)
yesterday = yesterday.strftime('%Y-%m-%d')


In [ ]:
keyword = input('Enter a topic or keyword, please:')



In [ ]:
print(keyword)

In [ ]:
#Create a list to append tweets 

import csv

tweet_list = []
maxTweets = 1000 #Open/create a file to append data to


csvFile = open(keyword +'-sentiment-' + now + '.csv', 'a', newline='', encoding='utf8')

#Use csv writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet',])

#use tweetsearchscraper to append tweets 

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' lang:en since:' +  yesterday + ' until:' + now + ' -filter:links -filter:replies').get_items()):
        if i > maxTweets :
            break
        csvWriter.writerow([tweet.id, tweet.date, tweet.content])
csvFile.close()

In [ ]:
tweet_df = pd.read_csv(keyword +'-sentiment-' + now + '.csv')


In [ ]:
tweet_df.head(5)

In [ ]:
tweet_df.shape

# 3. Cleaning the tweets

<br>We create a function to clean the tweets. We use regex (regular expressions) to remove @mentions, #hastags, hyperlinks, retweets, and many more. Finally, we apply the function to our tweets_to_df data frame and create a new column for the cleaned tweets<Br>

In [ ]:
def  cleanTweets(text):
    
    text = re.sub('@[A-Za-z0-9_]+', '', text) #removes @mentions
    text = re.sub('#','',text) #removes hastag '#' symbol
    text = re.sub('RT[\s]+','',text)
    text = re.sub('https?:\/\/\S+', '', text) 
    text = re.sub('\n',' ',text)
    return text


tweet_df["cleaned_tweets "] = tweet_df["tweet"].apply(cleanTweets)

tweet_df.head()

In [ ]:
tweet_df.to_csv('tweetsSentiment.csv') #write dataframe into csv file
savedTweets = pd.read_csv('tweetsSentiment.csv',index_col=0) #reads csv file
savedTweets.head(3)

# 4. Detect sentiments

<br> We create a function that gets the subjectivity and polarity of each tweet and saves them to new columns with the names Subjectivity and Polarity respectively.Using NLTK , 

<br>If a tweet has high subjectivity i.e. close to 1, it means the tweet contains more of a personal opinion than factual information.

<br>
The polarity score lies between (-1 to 1) where -1 identifies the most negative words and 1 identifies the most positive words <br>

In [ ]:
savedTweets['cleaned_tweets ']

In [ ]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity



In [ ]:
 #Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity






In [ ]:
savedTweets['Subjectivity'] = savedTweets['cleaned_tweets '].apply(getSubjectivity)
savedTweets['polarity'] = savedTweets['cleaned_tweets '].apply(getPolarity)

In [ ]:
#DropThe Unwanted Column

savedTweets = savedTweets.drop("tweet", axis = 1).head()

### Lets  determine if a tweet’s polarity is positive, neutral, or negative using a function

In [ ]:
#check negative, neutral and positive analysis

def getAnalysis(score):
    if score<0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'


savedTweets['Analysis'] = savedTweets['polarity'].apply(getAnalysis)

savedTweets.head(5)

In [ ]:
#Next, we count the total number of tweets for each polarity

savedTweets['Analysis'].value_counts()



 ## Visualization

In [ ]:
#Polarity graph to show sentiment distribution

from matplotlib.pyplot import colorbar
from seaborn import barplot


fig = plt.figure(figsize= (7,5))
savedTweets['Analysis'].value_counts().plot(kind = 'bar', colormap='Paired' )
#color = ['red','blue','orange']
plt.title("Sentiment distribution")
plt.ylabel("count")
plt.xlabel("Polarity")
plt.grid(False)
plt.show()


In [ ]:
#pie chart to show percentage distribution of polarity
#pie chart to show percentage distribution of polarity
fig = plt.figure(figsize=(7,7))
colors = ('green', 'grey', 'red')
wp={'linewidth':2, 'edgecolor': 'black'}
tags=savedTweets['Analysis'].value_counts()
explode = (0.1,0.1,0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors=colors, 
         startangle=90, wedgeprops=wp, explode=explode, label='')
plt.title('Distribution of polarity')


In [ ]:
#Plot the subjectivity and polarity on a scatter diagram

from turtle import color


plt.figure(figsize = (7,7))
for i in range (0,savedTweets.shape[0]):
    plt.scatter(savedTweets['polarity'],savedTweets['Subjectivity'],color = 'orange')

plt.title = 'subjectivity and polarity on a scatter diagram'
plt.xlabel ("Polarity")
plt.ylabel('Subjectivity')
plt.show()

# 5. Creating a word cloud for the tweets

<br>To understand which words have been used most in the tweets, we create a word cloud function for both positive and negative tweets.<br>

In [ ]:
def word_cloud(text):
    allwords = ''.join([tweets for tweets in text ])
    wordCloud = WordCloud(background_color= 'white',height= 700,width= 600 ,random_state= 30 ,max_font_size= 120).generate(allwords)
    plt.figure(figsize=(20,10))
    plt.imshow(wordCloud)
    plt.axis("off")
    plt.show()
tweets = savedTweets['cleaned_tweets ']
word_cloud(tweets)



In [ ]:
savedTweets.info()

In [ ]:
#create wordcloud for positive tweets
positiveTweets = savedTweets.loc[savedTweets['Analysis']== 'Positive','tweets']
word_cloud(positiveTweets)

In [ ]:
positiveTweets = savedTweets.loc[savedTweets['Analysis']== Negative, "cleaned_tweets"].item()
word_cloud(positiveTweets)

In [ ]:
# get the percentage of positive tweets
pTweets = savedTweets[savedTweets['Analysis']=='Positive']
pTweets = pTweets['cleaned_tweets']
percentage = round((pTweets.shape[0]/savedTweets.shape[0]) *100,1)
print('Percentage of positive tweets: {0}%'.format(percentage))

In [ ]:
# get the percentage of negative tweets
nTweets = savedTweets[savedTweets['Analysis']=='Negative']
nTweets = nTweets['cleanedTweets']
percentage = round((nTweets.shape[0]/savedTweets.shape[0]) *100,1)
print('Percentage of negative tweets: {0}%'.format(percentage))

In [ ]:
#break each tweet sentence into words
sentences = []
for word in savedTweets['cleaned_tweets']:
    sentences.append(word)
sentences

lines = list()
for line in sentences:
    words = line.split()
    for w in words:
        lines.append(w)
lines[:10]

In [ ]:
#stemming all the words to their root word
stemmer = SnowballStemmer(language='english')
stem=[]
for word in lines:
    stem.append(stemmer.stem(word))
stem[:20]

Next, we remove stop words which are the common words used in the English Language such as ‘on’, ‘the’, ‘is’ etc. We then group the rest together to their root words eg joined, joining, and joint are grouped together as a single word — join and save it to a new data frame df.

In [ ]:
#stemming all the words to their root word
stemmer = SnowballStemmer(language='english')
stem=[]
for word in lines:
    stem.append(stemmer.stem(word))
stem[:20]
#removes stopwords (very common words in a sentence)
stem2 = []
for word in stem:
    if word not in nlp.Defaults.stop_words:
        stem2.append(word)
#creates a new dataframe for the stem and shows the count of the most used words
df = pd.DataFrame(stem2)
df=df[0].value_counts()
df #shows the new dataframe

In [ ]:
#removes stopwords (very common words in a sentence)
stem2 = []
for word in stem:
    if word not in nlp.Defaults.stop_words:
        stem2.append(word)
#stem2[:20]

In [ ]:
#creates a new dataframe for the stem
df = pd.DataFrame(stem2)
df=df[0].value_counts()
df

In [ ]:
#plots the top 20 used words
df = df[:20]
plt.figure(figsize=(10,5))
sns.barplot(df.values, df.index, alpha=0.8)
plt.title('Top Words Overall')
plt.xlabel('Count of words', fontsize=12)
plt.ylabel('Words from Tweets', fontsize=12)
plt.show()